# Import Data and Transform for Publication

This is the notebook used to import the raw data from influxDB and transform it to a publishable format as csv file

Version = 1.0

Author = Patrick Ruoff

The environment is in conda_env.yml

In [1]:
from math import floor
from copy import deepcopy as dc

import pandas as pd
import numpy as np
from influxdb import DataFrameClient

import setup as se

In [2]:
def read_df_from_influxdb():
    """ Read data from influxDB instance

    Merge all data to one single pandas.DataFrame make the occupant numbers to features.

    :return: DataFrame with all the data
    """

    host = 'localhost'
    port = 8086
    user = 'pythonServer'
    reader = open('influxdb_password.txt')
    password = reader.read()
    dbname = 'openhab_db'
    client = DataFrameClient(host, port, user, password, dbname)

    print('Starting import from {} between {} and {}'.format(
        dbname, se.startTime, se.endTime))
    # NOTE: beware that timestamps now are +00:00 -> different from Grafana
    data = pd.DataFrame()

    if not np.shape(se.inputSeries) == np.shape(se.outputSeries):
        print('InputSerieses and se.outputSerieses must have the same number of entries! '
              'Found: {}, {}'
              .format(np.shape(se.inputSeries), np.shape(se.outputSeries)))

    start_time_unix = "{}000000000".format(int(pd.to_datetime(se.startTime).timestamp()))
    end_time_unix = "{}000000000".format(int(pd.to_datetime(se.endTime).timestamp()))
    # first is different from others
    data_frames = {}
    i = 0
    while i < np.shape(se.inputSeries)[0]:
        try:
            data_frames[se.outputSeries[i]] = client.query(
                "SELECT * FROM {} WHERE time > {} AND time < {}".format(
                    se.inputSeries[i], start_time_unix, end_time_unix))[se.inputSeries[i]]
        except KeyError:
            print('Input series {} has no entries in specified time frame'.format(
                se.inputSeries[i]))
            if se.outputSeries[i] in se.seriesEnvironment:
                se.seriesEnvironment.remove(se.outputSeries[i])
                if se.outputSeries[i] in se.categoricalSeries:
                    se.categoricalSeries.remove(se.outputSeries[i])
            elif se.outputSeries[i] in se.seriesBiosignal:
                if se.outputSeries[i][:2] in se.availableUsers:
                    se.availableUsers.remove(se.outputSeries[i][:2])
                se.seriesBiosignal.remove(se.outputSeries[i])
            else:
                se.seriesVote.remove(se.outputSeries[i])
            del se.inputSeries[i]
            del se.outputSeries[i]
            continue

        current_df = pd.DataFrame(data_frames[se.outputSeries[i]])
        current_df.columns = [se.outputSeries[i]]
        # round to 30 seconds
        new_indices = []
        for ind in data_frames[se.outputSeries[i]][se.outputSeries[i]].index:
            ind = ind.replace(second=floor(ind.second/30)*30, microsecond=0, nanosecond=0)
            new_indices.append(ind)
        current_df.index = new_indices
        data_frames[se.outputSeries[i]] = current_df
        i += 1

    # update userSerieses
    for user in se.availableUsers:
        se.userSeries[user] = []
        for series in se.outputSeries:
            if user in series:
                se.userSeries[user].append(series)

    # unite to one df
    data = pd.DataFrame([], index=pd.date_range(
        start=se.startTime, end=se.endTime - pd.Timedelta('30s'), freq='30s'))
    for i in (np.arange(np.shape(se.outputSeries)[0])):
        # handle duplicates
        is_duplicated_array = data_frames[se.outputSeries[i]].index.duplicated(False)
        if any(is_duplicated_array):
            data_frames[se.outputSeries[i]].index.name = 'index'
            print('duplicated indices:\n',
                  data_frames[se.outputSeries[i]][is_duplicated_array])
            duplicated_indices = np.unique(
                (data_frames[se.outputSeries[i]][is_duplicated_array]).index.to_numpy())
            if data_frames[se.outputSeries[i]]._get_numeric_data().columns.empty:
                # for categorical data
                data_frames[se.outputSeries[i]] = \
                    data_frames[se.outputSeries[i]].groupby('index').last()
            else:
                if se.outputSeries[i] in se.seriesEnvironment or \
                        se.outputSeries[i] in se.seriesBiosignal:
                    # for numerical sensor data
                    data_frames[se.outputSeries[i]] = \
                        data_frames[se.outputSeries[i]].groupby('index').mean()
                else:
                    # for numerical voting data use last voting
                    data_frames[se.outputSeries[i]] = \
                        data_frames[se.outputSeries[i]].groupby('index').last()
            print('\nare merged to:\n',
                  data_frames[se.outputSeries[i]].loc[duplicated_indices])
        # used to be data[se.outputSerieses[i]] = data_frames[se.outputSerieses[i]]
        # but that limits the time span to the first series' last value
        data = data.merge(
            right=data_frames[se.outputSeries[i]], how='outer',
            left_index=True, right_index=True)

    # make categorical features strings
    for i, series in enumerate(se.categoricalSeries):
        if series in data.columns:
            data[series] = data[series].astype(str)
            # nan values stay np.nan
            data[series] = data[series].replace('nan', np.nan)
        else:
            del se.categoricalSeries[i]
    data.index.name = 'index'

    print('Data from {} to {} in influxDB is of size {}'.format(se.startTime, se.endTime, np.shape(data)))
    return data


In [3]:
def _apply_manual_fixes(data):
    # 14.11.2019 10:40: Exchange U3 and Corridor Sensor
    # Sensor that was U3_ESP8266 until 14.11.2019 10:40 and became
    # corridor_ESP8266 afterwards has many outliers
    changing_ts = pd.Timestamp('2019-11-14 10:40:00-05:00')
    if changing_ts in data.index:
        for series in ['temp', 'hum']:
            corridor_string = 'corridor_{}'.format(series)
            user_string = 'U3_desk_{}'.format(series)
            if user_string in data and corridor_string in data:
                intermediate = dc(
                    data.loc[data.index <= changing_ts,
                         corridor_string].values)
                early_user_frame = data.loc[data.index <= changing_ts,
                                            user_string]
                data.loc[
                    data.index <= changing_ts, corridor_string] = early_user_frame.values
                data.loc[data.index <= changing_ts, user_string] = intermediate
                late_corridor_frame = data.loc[data.index > changing_ts, corridor_string]
                data.loc[data.index > changing_ts, corridor_string] = \
                    late_corridor_frame.values

    # replaceInterals = [[gapStart, gapEnd, seriesTarget, seriesSource, offset]]
    for intervalInfo in se.replaceIntervals:
        if intervalInfo[0] in data.index and intervalInfo[1] in data.index:
            data = _preplace_series_by_series_in_timeframe(data, [intervalInfo[0], intervalInfo[1]],
                                                           series_target=intervalInfo[2],
                                                           series_source=intervalInfo[3],
                                                           offset=intervalInfo[4])
    # afterwards IW_general_humidity not of any use
    # since sensor broke in the middle of data collection
    if 'IW_general_hum' in se.seriesEnvironment:
        data = data.drop(columns=['IW_general_hum'])
        se.seriesEnvironment.remove('IW_general_hum')
        se.outputSeries.remove('IW_general_hum')
    return data


In [4]:
def _preplace_series_by_series_in_timeframe(data, timeframe, series_target,
                                            series_source, offset):
    if series_source in data.columns and series_target in data.columns:
        data.loc[
            (data[series_target].index > timeframe[0]) &
            (data[series_target].index < timeframe[1]),
            series_target
        ] = data.loc[
                    (data[series_source].index > timeframe[0]) &
                    (data[series_source].index < timeframe[1]),
                    series_source
                ].copy().values + offset
    return data

In [5]:
se.init()
se.startTime = pd.Timestamp('2020-02-18 17:00:00-05:00')
se.endTime = pd.Timestamp('2020-03-06 17:00:00-05:00')
data = read_df_from_influxdb()

Starting import from openhab_db between 2020-02-18 17:00:00-05:00 and 2020-03-06 17:00:00-05:00
Input series infrared_switch_U1 has no entries in specified time frame
duplicated indices:
                            U1_desk_hum
index                                 
2020-02-24 19:26:30+00:00    25.900000
2020-02-24 19:26:30+00:00    25.900000
2020-03-02 07:31:30+00:00    23.100000
2020-03-02 07:31:30+00:00    23.100000
2020-03-02 07:37:00+00:00    23.100000
2020-03-02 07:37:00+00:00    23.200001
2020-03-03 14:52:30+00:00    39.099998
2020-03-03 14:52:30+00:00    39.099998
2020-03-03 14:54:30+00:00    39.099998
2020-03-03 14:54:30+00:00    39.099998

are merged to:
                            U1_desk_hum
index                                 
2020-02-24 19:26:30+00:00    25.900000
2020-03-02 07:31:30+00:00    23.100000
2020-03-02 07:37:00+00:00    23.150001
2020-03-03 14:52:30+00:00    39.099998
2020-03-03 14:54:30+00:00    39.099998
duplicated indices:
                            U1_des

duplicated indices:
                            U2_thermal_comfort_vote
index                                             
2020-02-19 17:13:30+00:00                        1
2020-02-19 17:13:30+00:00                        2

are merged to:
                            U2_thermal_comfort_vote
index                                             
2020-02-19 17:13:30+00:00                        2
duplicated indices:
                            U2_thermal_sensation_vote
index                                               
2020-02-19 17:13:30+00:00                          2
2020-02-19 17:13:30+00:00                          1

are merged to:
                            U2_thermal_sensation_vote
index                                               
2020-02-19 17:13:30+00:00                          1
duplicated indices:
                            U2_infrared_heater_switch
index                                               
2020-02-28 19:36:00+00:00                          1
2020-02-28 19:36

Data from 2020-02-18 17:00:00-05:00 to 2020-03-06 17:00:00-05:00 in influxDB is of size (48960, 40)


In [6]:
data = _apply_manual_fixes(data)

In [15]:
cols = list(data.columns)

In [28]:
cols = cols[:31] + [cols[-1]] + cols[31:-1]

In [29]:
data = data[cols]

In [30]:
data.describe()

floor_CO2    floor_temp     floor_hum  outside_cloudiness  \
count  48951.000000  48951.000000  48951.000000        48951.000000   
mean     613.724144     70.891068     25.527994           49.242712   
std       93.039986      1.712820      0.000000           41.492692   
min      483.799927     66.459190     25.527994            1.000000   
25%      537.399902     69.850388     25.527994            1.000000   
50%      596.399902     71.232788     25.527994           75.000000   
75%      666.199951     72.240784     25.527994           90.000000   
max      987.399902     74.494385     25.527994           90.000000   

       outside_humidity  outside_pressure  outside_temperature  \
count      48951.000000      48951.000000         48951.000000   
mean          62.851443         30.015488            35.809698   
std           21.470623          0.260116            10.521490   
min           20.000000         29.441399            11.084000   
25%           45.000000         29.825289            26.420000   
50%           63.000000         29.943409            36.230000   
75%           79.000000         30.179649            45.104000   
max          100.000000         30.563538            56.786000   

       outside_wind_speed   U1_desk_hum  U1_desk_temp  ...  \
count        48951.000000  48722.000000  48722.000000  ...   
mean             8.750697     27.600286     71.318150  ...   
std              4.377772      5.873159      1.841161  ...   
min              0.872404     18.600000     66.739998  ...   
25%              5.816030     22.500000     70.160004  ...   
50%              8.052964     25.700001     71.419998  ...   
75%             11.408366     31.299999     72.500000  ...   
max             26.395827     41.599998     75.739998  ...   

       U1_infrared_heater_switch  U2_thermal_comfort_vote  \
count                        0.0                68.000000   
mean                         NaN                 1.294118   
std                          NaN                 0.490471   
min                          NaN                 0.000000   
25%                          NaN                 1.000000   
50%                          NaN                 1.000000   
75%                          NaN                 2.000000   
max                          NaN                 2.000000   

       U2_thermal_sensation_vote  U2_infrared_heater_switch  \
count                   68.00000                  25.000000   
mean                     1.50000                   0.440000   
std                      0.70181                   0.506623   
min                      0.00000                   0.000000   
25%                      1.00000                   0.000000   
50%                      2.00000                   0.000000   
75%                      2.00000                   1.000000   
max                      3.00000                   1.000000   

       U3_thermal_comfort_vote  U3_thermal_sensation_vote  \
count               349.000000                 349.000000   
mean                  1.131805                   1.793696   
std                   0.378810                   0.452154   
min                   0.000000                   0.000000   
25%                   1.000000                   2.000000   
50%                   1.000000                   2.000000   
75%                   1.000000                   2.000000   
max                   2.000000                   2.000000   

       U3_infrared_heater_switch  U1_last_command_encoded  \
count                  66.000000              9645.000000   
mean                    0.469697                -0.588284   
std                     0.502905                 0.499697   
min                     0.000000                -3.000000   
25%                     0.000000                -1.000000   
50%                     0.000000                -1.000000   
75%                     1.000000                 0.000000   
max                     1.000000                 0.000000   

The sensor for `floor_hum` was stuck on one value for the whole period of observation

In [ ]:
del data['floor_hum']

In [31]:
data.to_csv('temperature_control_phase.csv')